# <center> 2차 크롤링 - <원티드> </center>

##### 링크들 들어가서 각각 정보 수집, 드론

In [1]:
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import urllib


In [16]:
options = webdriver.ChromeOptions()

url = "https://www.wanted.co.kr/jobsfeed"
driver = webdriver.Chrome(service=Service("../../driver/chromedriver"),
                          options=options)

driver.get(url)

In [3]:
driver.get(url)

In [17]:
nav_element = driver.find_element(By.CLASS_NAME, "Aside_asideList__3WNyu")
search_element = nav_element.find_element(By.CLASS_NAME, "SvgIcon_SvgIcon__root__8vwon")

search_element.click()

In [18]:
search_doc = driver.find_element(By.CLASS_NAME, "SearchInput_SearchInput__gySrv")
search_doc.send_keys('드론')
search_doc.send_keys(Keys.ENTER)


In [19]:
wait = WebDriverWait(driver, 10)
position_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(), "포지션")]')))
position_tab.click()

In [20]:
simple_list = []
wait = WebDriverWait(driver, 10)  

simple_name_elements = []  

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    try:
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')) > len(simple_name_elements))
    except TimeoutException:
        break

    new_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')
    simple_name_elements.extend(new_elements)

    # StaleElementReferenceException을 처리하기 위해 엘리먼트 수의 변경을 확인
    try:
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')) > len(simple_name_elements))
    except TimeoutException:
        break

    if len(simple_name_elements) >= 20:
        break


simple_names = [element.text.strip() for element in simple_name_elements]
print(simple_names)
print(len(simple_names))

['', '서울 · 한국']
2


In [21]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time

# 기업이름 요소 초기 로드
company_name_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')
num_elements_before_scroll = len(company_name_elements)

while True:
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2) 

    try:
        # 새로운 회사명 요소들이 로드될 때까지 대기 (타임아웃 10초)
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')) > num_elements_before_scroll)
    except TimeoutException as ex:
        break

    # 현재 로드된 회사명 요소들 가져오기
    company_name_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')
    num_elements_before_scroll = len(company_name_elements)

    if num_elements_before_scroll >= 20:  
        break

company_names = []
href_lists = []

href_elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-attribute-id="position__click"]')

# 가져온 데이터 중에서 조건에 맞는 것들만 필터링
for i, element in enumerate(company_name_elements):
    simple_name = simple_names[min(i, len(simple_names) - 1)]
    company_name_text = element.text.strip()


    # simple_name에 '서울 · 한국' 또는 '경기 · 한국'이 포함된 경우에만 추가
    if '서울 · 한국' in simple_name or '경기 · 한국' in simple_name:
        company_names.append(company_name_text)
        href_value = href_elements[i].get_attribute('href')
        href_lists.append(href_value)



In [22]:
#직무(담당업무 열)가져오기
job_title_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM') 
num_elements_before_scroll = len(company_name_elements)

while True:
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2) 

    try:
        # 새로운 회사명 요소들이 로드될 때까지 대기 (타임아웃 10초)
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM')) > num_elements_before_scroll)
    except TimeoutException as ex:
        break

    job_title_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM')
    num_elements_before_scroll = len(company_name_elements)

    if num_elements_before_scroll >= 20:  
        break

job_titles = []

for i, element in enumerate(job_title_elements):
    # simple_names 리스트에서 현재 인덱스에 해당하는 값 가져오기
    simple_name = simple_names[min(i, len(simple_names)-1)]

    # 만약 값이 '서울'이나 '경기'로 시작하면 추가
    if simple_name.startswith('서울') or simple_name.startswith('경기'):
        job_titles.append(job_title_elements[i].text.strip())  # 수정된 부분

In [23]:

# 기업주소 리스트 초기화
address_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    # 페이지 이동
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 기업 주소를 포함한 <section> 요소 찾기
        address_div = soup.find('section', class_='JobWorkPlace_className__ra6rp')

        if address_div:
            # <section> 내부의 주소 정보를 찾아서 리스트에 추가
            address_items = address_div.find_all('span', class_='body')
            address = [address_item.text.strip() for address_item in address_items]

            # URL과 해당하는 주소 리스트를 한 리스트에 추가
            address_list.append(address)
            break  # 주소를 찾았으므로 루프 종료

        # 스크롤을 크게 아래로 내림 (300픽셀씩)
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)  # 스크롤 후 충분한 시간 대기



In [24]:
# address_list에 있는 각 기업주소에 대해 처리
for i in range(len(address_list)):
    # 첫 번째 요소를 제외한 나머지 주소로 갱신
    address_list[i] = address_list[i][1:]

# 리스트 출력
for address in address_list:
    print(f'{address}\n')

print(len(address_list))

['서울시 강남구 논현로 417 화원빌딩 4, 5층']

1


In [25]:
#연봉
salary_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    # 페이지 이동
    driver.get(url)


    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 기술 스택을 포함한 <div> 요소 찾기
        salary_div = soup.find('div', class_='Tags_tagsClass__mvehZ')

        if salary_div:
            salary_items = salary_div.find_all('li')
            first_li = salary_items[0] if salary_items else None

            if first_li and "연봉" in first_li.text:
                salaries = [salary_item.text.strip().replace("#", "") for salary_item in salary_items]
                salary_list.append(salaries[0])
                break
            else:
                salary_list.append(["-"])
                break


     
# 리스트 출력
for i in salary_list:
    print(f'{i}\n')

print(len(salary_list))

['-']

1


In [26]:
# 자격요건 텍스트를 저장할 리스트 초기화
qualifications_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "자격요건" 부분을 추출
        qualifications_header = soup.find('h3', string='자격요건')

        if qualifications_header:
            # "자격요건" 부분이 포함된 <span> 요소 찾기
            qualifications_span = qualifications_header.find_next('p').find('span')

            if qualifications_span:
                # <span> 요소에서 텍스트 추출
                qualifications_text = qualifications_span.get_text(strip=True)
                # 리스트에 자격요건 추가
                qualifications_list.append(qualifications_text)
                break  # 자격요건을 찾았으므로 루프 종료
            else:
                print('자격요건이 포함된 <span> 요소를 찾을 수 없습니다.')
        else:
            # 스크롤을 계속해서 아래로 내림
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

for qualifications in qualifications_list:
    print(f'{qualifications}\n')

print(len(qualifications_list))

3년 이상의 S/W 개발 경험 (분야 무관)다양한 프로그래밍 언어를 자유롭게 활용 가능하신 분새로운 도메인에 대한 학습 능력 및 문제해결 역량을 갖추신 분주어진 문제를 풀기 위해 사업팀과 소통하며 고객을 위한 공학적인 의사 결정을 주도적으로 하실 수 있는 분

1


In [27]:
#담당업무
work_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "주요업무" 부분을 추출
        work_header = soup.find('h3', string='주요업무')

        if work_header:
            work_span = work_header.find_next('p').find('span')

            if work_span:
                # <span> 요소에서 텍스트 추출
                work_text = work_span.get_text(strip=True)
                work_list.append(work_text)
                break  
            else:
                print('담당업무가 포함된 <span> 요소를 찾을 수 없습니다.')
        else:
            # 스크롤을 계속해서 아래로 내림
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

for work in work_list:
    print(f'{work}\n')

print(len(work_list))

AI 드론을 원격에서 관리하는 관제 S/W 개발

1


In [28]:
#우대사항
preferential_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "주요업무" 부분을 추출
        preferential_header = soup.find('h3', string='우대사항')

        if preferential_header:
            preferential_span = preferential_header.find_next('p').find('span')

            if preferential_span:
                # <span> 요소에서 텍스트 추출
                preferential_text = preferential_span.get_text(strip=True)
                preferential_list.append(preferential_text)
                break  
            else:
                print('우대사항이 포함된 <span> 요소를 찾을 수 없습니다.')
        else:
            # 스크롤을 계속해서 아래로 내림
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

for preferential in preferential_list:
    print(f'{preferential}\n')

print(len(preferential_list))

H/W를 원격에서 모니터링하고 제어하는 S/W 개발 경험자스타트업 경험자【 고용형태 】• 정규직 (수습기간 3개월 포함)【 채용절차 】• 서류전형 코딩테스트 1,2차 면접  최종합격• 지원서 : 자유양식• 그 외 포트폴리오 등 업무역량을 알 수 있는 자료가 있다면 자유롭게 보내주세요:-)• 최종합격 전 처우 협의가 진행됩니다.• 각 절차별 합격자에 한해 다음 절차가 진행됩니다.

1


In [14]:
driver.quit()

In [31]:
# 같은 인덱스끼리 합치기(자격요건 + 우대사항 + 담당업무)
required_list = [qual + pref + work for qual, pref, work in zip(qualifications_list, preferential_list, work_list)]
for item in required_list:
    print(item)

3년 이상의 S/W 개발 경험 (분야 무관)다양한 프로그래밍 언어를 자유롭게 활용 가능하신 분새로운 도메인에 대한 학습 능력 및 문제해결 역량을 갖추신 분주어진 문제를 풀기 위해 사업팀과 소통하며 고객을 위한 공학적인 의사 결정을 주도적으로 하실 수 있는 분H/W를 원격에서 모니터링하고 제어하는 S/W 개발 경험자스타트업 경험자【 고용형태 】• 정규직 (수습기간 3개월 포함)【 채용절차 】• 서류전형 코딩테스트 1,2차 면접  최종합격• 지원서 : 자유양식• 그 외 포트폴리오 등 업무역량을 알 수 있는 자료가 있다면 자유롭게 보내주세요:-)• 최종합격 전 처우 협의가 진행됩니다.• 각 절차별 합격자에 한해 다음 절차가 진행됩니다.AI 드론을 원격에서 관리하는 관제 S/W 개발


In [32]:
keyword_list = ["드론"] * len(company_names)

print(len(keyword_list))
print(keyword_list)

1
['드론']


In [33]:
import pandas as pd
# 데이터프레임 생성
data = {
    '기업이름': company_names,
    '연봉': salary_list,
    '기업주소': address_list,
    '키워드': keyword_list,
    '담당업무': job_titles,
    '필요역량': required_list
}
df = pd.DataFrame(data)
# 데이터프레임 출력
df.head()

,기업이름,연봉,기업주소,키워드,담당업무,필요역량
0,니어스랩,[-],"[서울시 강남구 논현로 417 화원빌딩 4, 5층]",드론,소프트웨어 엔지니어 팀장 (드론 원격 관제 S/W 개발),3년 이상의 S/W 개발 경험 (분야 무관)다양한 프로그래밍 언어를 자유롭게 활용 ...


In [34]:
# • 문자를 ,으로 바꾸기
df['담당업무'] = df['담당업무'].str.replace('•', ',')
df['필요역량'] = df['필요역량'].str.replace('•', ',')

# 작은 따옴표 및 괄호 제거
df['연봉'] = df['연봉'].astype(str).str.replace('[', '').str.replace(']', '').str.strip("'")
df['기업주소'] = df['기업주소'].astype(str).str.replace('[', '').str.replace(']', '').str.strip("'")
df.head()

,기업이름,연봉,기업주소,키워드,담당업무,필요역량
0,니어스랩,-,"서울시 강남구 논현로 417 화원빌딩 4, 5층",드론,소프트웨어 엔지니어 팀장 (드론 원격 관제 S/W 개발),3년 이상의 S/W 개발 경험 (분야 무관)다양한 프로그래밍 언어를 자유롭게 활용 ...


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기업이름    1 non-null      object
 1   연봉      1 non-null      object
 2   기업주소    1 non-null      object
 3   키워드     1 non-null      object
 4   담당업무    1 non-null      object
 5   필요역량    1 non-null      object
dtypes: object(6)
memory usage: 176.0+ bytes


In [36]:
import re

# 경력 정보 추출을 위한 정규표현식
career_pattern = re.compile(r'(\d+년.*?)')

# DataFrame의 각 행에 대해 경력 정보 추출 및 새로운 열에 추가
df['고용형태'] = df['필요역량'].apply(lambda x: career_pattern.findall(x)[0] if career_pattern.findall(x) else '-')

# 결과 출력
print(df[['고용형태']])

  고용형태
0   3년


In [37]:
df = df[['기업이름','고용형태','연봉','기업주소','키워드','담당업무','필요역량' ]]

In [38]:
df.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,니어스랩,3년,-,"서울시 강남구 논현로 417 화원빌딩 4, 5층",드론,소프트웨어 엔지니어 팀장 (드론 원격 관제 S/W 개발),3년 이상의 S/W 개발 경험 (분야 무관)다양한 프로그래밍 언어를 자유롭게 활용 ...


In [39]:
import csv

df.to_csv('output1.csv', index=False)